## 4.2.1 python作用域基础

### 4.2.1.1 作用域细节
![](.4_images/ff2dd26f.png)


### 4.2.1.2 变量名解析
![](.4_images/12b5c0c0.png)
![](.4_images/48e3eba8.png)

### 4.2.1.3 作用域实例

In [1]:
# x是全局作用域
x = 99
# func也是全局作用域
def func(y):
    # z和y都是局部作用域
    z = x+y
    return z

print(func(1))

100


### 4.2.1.4 内置作用域

In [2]:
# 其实python所有的内置关键词都是通过内置作用域来实现的
import builtins
print(dir(builtins))

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'SystemError

In [3]:
# 下面这两个是完全等价的
print(zip is builtins.zip)

True


In [4]:
# 在函数里面修改全局变量无效
x = 88
def func():
    x = 99
func()
print(x)

88


## 4.2.2 global语句

In [5]:
# 上面这个问题可以通过global来解决
x = 88
def func():
    global x
    x =99
func()
print(x)

99


In [6]:
# 没有变量会自动创建变量
x,y = 1,3
def all_global():
    global z
    z = x+y
all_global()
print(z)

4


### 4.2.2.1 最小化全局变量

In [ ]:
# 针对下面这个模块文件，过度使用全局变量会导致我们的函数难以理解
x = 99
def func1():
    global x
    x = 88
def func2():
    global x
    x = 77

### 4.2.2.2 最小化跨文件修改

In [1]:
import first
print(first.X)
# 不要这样跨文件去修改遍历，会导致模块太耦合了
first.X = 100

99


In [2]:
# 最好的方式就是使用模块的函数去修改
first.set_x(66)
print(first.X)

66


### 4.2.2.3 其他访问全局变量的方式

In [ ]:
# 比如我们定义了一个thismod.py 内容如下
var = 99
# 那么修改的方法有下面这几种

![](.4_images/6875989b.png)

## 4.2.3 作用域和嵌套参数

In [3]:
x = 99
def f1():
    x = 88
    def f2():
        print(x)
    f2()
f1()

88


### 4.2.3.3 工厂函数(闭包)

In [4]:
# 下面我们定义一个简单的工厂函数
def maker(N):
    def action(X):
        return X ** N
    return action

# 然后我们可以创造出一个action
f = maker(2)
# 这样我们就可以计算2的任意次方了
print(f(2))
print(f(4))

4
16


In [5]:
# 这个时候我们在定义一个
g = maker(3)
# 实际上不会相互影响
print(g(4))
print(f(4))

64
16


In [6]:
# 这个方法其实可以简化一下
def maker(N):
    return lambda X:X**N
h = maker(3)
print(h(4))

64


### 4.2.3.4 使用默认值

In [8]:
def func():
    x = 4
    action = (lambda m:x**m)
    return action

x = func()
print(x(3))

64


In [9]:
# 我们可以创建一个函数列表，但是下面这个所有函数的i值都是4
def makeActions():
    acts = []
    for i in range(5):
        acts.append(lambda x:i**x)
    return acts

acts = makeActions()
print(acts[0])
print(acts[0](2))
print(acts[2](2))
print(acts[4](2))

<function makeActions.<locals>.<lambda> at 0x000001F522C2AC18>
16
16
16


In [11]:
# 为了避免这个问题，我们可以使用默认值
def makeActions():
    acts = []
    for i in range(5):
        acts.append(lambda x,i=i:i**x)
    return acts
acts = makeActions()
print(acts[0](2))
print(acts[2](2))
print(acts[4](2))

0
4
16


## 4.2.4 nonlocal

### 4.2.4.1 nonlocal基础

In [13]:
# 一般情况下，我们只能获取外层def的属性，但是不能修改
def tester(start):
    state = start
    def nested(label):
        print(label, state)
        # state += 1
    return nested

F = tester(0)
F('spam')

spam 0


In [14]:
# 如果想修改的话，需要使用state来修改
def tester(start):
    state = start
    def nested(label):
        nonlocal state
        print(label, state)
        state += 1
    return nested

F = tester(0)
F('spam')
F('ham')
F('egge')

spam 0
ham 1
egge 2


In [15]:
# 可以多次调用闭包
G = tester(42)
G('spam')
G('egg')
F('bacon')

spam 42
egg 43
bacon 3


In [16]:
# nonlocal和global是有区别的,nonlocal必须是在外层def作用域被赋值过
def tester(start):
    def nested(label):
        nonlocal state
        state = 0
        print(label, state)
    return nested

SyntaxError: no binding for nonlocal 'state' found (3731199813.py, line 4)

In [18]:
# 但是global不需要重新定义
def tester(start):
    def nested(label):
        global state
        state = 0
        print(label, state)
    return nested
f = tester(0)
f('abc')
print(state)

abc 0
0


In [19]:
# nonlocal不能只有def之外的变量
spam = 99
def tester():
    def nested():
        nonlocal spam
        print('current=', spam)
        spam += 1
    return nested

SyntaxError: no binding for nonlocal 'spam' found (3414806883.py, line 5)

### 4.2.4.2 nonlocal应用

### 4.2.4.3 变量状态

In [20]:
# nonlocal是不能直接访问的
def tester(start):
    state = start
    def nested(label):
        nonlocal state
        print(label, state)
        state += 1
    return nested

F = tester(0)
F('spam')
print(F.state)

spam 0


AttributeError: 'function' object has no attribute 'state'

### 4.2.4.4 全局变量

In [23]:
# global只有一份副本
def tester(start):
    global state
    state = start
    def nested(label):
        global state
        state += 1
        print(label, state)
    return nested
f = tester(0)
f('spam')
f('eggs')
G = tester(42)
G('toast')
G('bacon')
f('ham')

spam 1
eggs 2
toast 43
bacon 44
ham 45


### 4.2.4.5 函数的属性

In [26]:
# 我们可以直接给函数设置属性
def tester(start):
    def nested(label):
        print(label, nested.state)
        nested.state += 1
    nested.state = start
    return nested

f = tester(0)
f('spam')
f('ham')
print(f.state)
g = tester(42)
g('eggs')
print(f.state)
print(g.state)

spam 0
ham 1
2
eggs 42
2
43


### 4.2.4.6 定制open

In [28]:
import builtins
def makeopen(id):
    original = builtins.open
    def custom(*kargs, **pargs):
        print("自定义打开文件 %r" % id, kargs, pargs)
        return original(*kargs, **pargs)
    builtins.open = custom

print(open("first.py").read())
makeopen('spam')
print(open("first.py").read())

X = 99


def set_x(new):
    global X
    X = new

自定义打开文件 'spam' ('first.py',) {}
X = 99


def set_x(new):
    global X
    X = new

